This function computes the gradient of the velocity-field $ \mathbf{v}(\mathbf{x}, t) $ using an auxiliary meshgrid.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| t | float | time |
| x | array (2,) | $ \mathbf{x} $ |
| X | array (Ny, Nx) | X-meshgrid|
| Y | array (Ny, Nx) | Y-meshgrid|
| Interpolant | list (2,) | Interpolant[0]: Interpolant object for $ u((\mathbf{x}, t))  $ <br /> Interpolant[1]: Interpolant object for $ v((\mathbf{x}, t))  $. <br /> <br /> The interpolation, however, works only <br /> if the particle is in the defined flow domain. <br /> At the boundary of the domain, <br /> the interpolant is not defined.|
| periodic | list (2,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br />|
| defined_domain | array (Ny, Nx) | points on the meshgrid where velocity field is defined |
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| dt_data | float | time spacing of the velocity data |
| delta_data | list (2,) | delta[0]: x-grid spacing of the velocity data <br /> delta[1]: y-grid spacing of the velocity data|
| aux_grid | list (2,) | aux_grid[0]: dx_auxiliary spacing <br /> aux_grid[1]: dy_auxiliary spacing |
| grad_vel | array (2,2) | $ \nabla \mathbf{v}(\mathbf{x}, t) $ |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add integration folder to current working path
sys.path.append(parent_directory+"/integration")

In [2]:
# Import numpy as np
import numpy as np

# Import function that computes velocity in the domain
from ipynb.fs.defs.velocity import velocity

In [3]:
def gradient_velocity(t, x, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta_data, aux_grid):
        
    # auxiliary grid spacing
    rho_x = aux_grid[0]
    rho_y = aux_grid[1]
    
    grad_vel = np.zeros((2, 2))
    
    # velocity on the auxiliary meshgrid
    xL = x-np.array([1, 0])*rho_x
    vLend = velocity(t, xL, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta_data)
        
    xR = x+np.array([1, 0])*rho_x
    vRend = velocity(t, xR, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta_data)
        
    xD = x-np.array([0, 1])*rho_y
    vDend = velocity(t, xD, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta_data)
        
    xU = x+np.array([0, 1])*rho_y
    vUend = velocity(t, xU, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta_data)
    
    grad_vel[0, 0] = (vRend[0]-vLend[0])/(2*rho_x)
    grad_vel[1, 0] = (vRend[1]-vLend[1])/(2*rho_x)
    grad_vel[0, 1] = (vUend[0]-vDend[0])/(2*rho_y)
    grad_vel[1, 1] = (vUend[1]-vDend[1])/(2*rho_y)
                
    return grad_vel